# What we're basically doing:

1. Loading a bunch of "earnings call" pdfs containing financial sentences, segmenting them to sentences.

2. Replacing words that are contained in lexicons/dictionaries witht their lexicon title throughout all sentences.
    (example: "profits gone up 3% last quarter." -> " LagInd gone dir:up 3% last quarter.")
    
3. Training a word2vec embedding utilizing these transformed sentences(we feed it by paragraphs)

4. Built a function that transforms a 2d matrix of embedding(representing a sentence), to a 1d vector(by averaging throughout all the word vectors). This way we obtain a sentence vector(a vector representing every sentence).

5. We need these sentence vectors as it allows us to classify per sentence basis, and that it is convenient for us to layer various downstream classifiers on top of it as they now in a very structured manner.


# Importing basic libraries 

In [1]:
import pandas as pd
import string
import os
import re

# Loading tagging dictionaries

In [2]:
#directionality
labelled_data = []
labelled_direction = []    
with open("resources/tagging_lexicons/Directionality.txt", 'r', encoding="ISO-8859-1") as f:
    labelled_data = f.readlines()   
for pairs in labelled_data:
    labelled_direction += [(pairs[:-1].strip().split(" ")[0].strip(),pairs[:-1].strip().split(" ")[1].strip())]

In [3]:
#positive
labelled_data = []
labelled_positive = []    
with open("resources/tagging_lexicons/LM_positive.txt", 'r', encoding="ISO-8859-1") as f:
    labelled_data = f.readlines()   
for words in labelled_data:
    labelled_positive += [words[:-1].strip()]
    labelled_positive = [x.lower() for x in labelled_positive]
labelled_positive = sorted(labelled_positive) 

In [4]:
#negative
labelled_data = []
labelled_negative = []    
with open("resources/tagging_lexicons/LM_negative.txt", 'r', encoding="ISO-8859-1") as f:
    labelled_data = f.readlines()   
for words in labelled_data:
    labelled_negative += [words[:-1].strip()]
    labelled_negative = [x.lower() for x in labelled_negative]
labelled_negative = sorted(labelled_negative) 

In [5]:
#leading
labelled_data = []
labelled_leading = []    
with open("resources/tagging_lexicons/Leading Indicators.txt", 'r', encoding="ISO-8859-1") as f:
    labelled_data = f.readlines()   

for words in labelled_data:
    labelled_leading += [words[:-1].strip()]
labelled_leading = sorted(labelled_leading) 

In [6]:
#lagging
labelled_data = []
labelled_lagging = []    
with open("resources/tagging_lexicons/Lagging Indicators.txt", 'r', encoding="ISO-8859-1") as f:
    labelled_data = f.readlines()   

for words in labelled_data:
    labelled_lagging += [words[:-1].strip()]
labelled_lagging = sorted(labelled_lagging)

In [7]:
#uncertainty
labelled_data = []
labelled_uncertainty = []    
with open("resources/tagging_lexicons/LM_uncertainty.txt", 'r', encoding="ISO-8859-1") as f:
    labelled_data = f.readlines()
for words in labelled_data:
    labelled_uncertainty += [words[:-1].strip().split(" ")]
final_labelled_uncertainty = []    
for i in labelled_uncertainty:
    final_labelled_uncertainty += i
    
final_labelled_uncertainty = [x.lower() for x in final_labelled_uncertainty][:-1]

In [8]:
#constraining
labelled_data = []
labelled_constraining = []    
with open("resources/tagging_lexicons/LM_constraining.txt", 'r', encoding="ISO-8859-1") as f:
    labelled_data = f.readlines()
for words in labelled_data:
    labelled_constraining += [words[:-1].strip().split(" ")]
final_labelled_constraining = []    
for i in labelled_constraining:
    final_labelled_constraining += i
final_labelled_constraining = [x.lower() for x in final_labelled_constraining]

In [9]:
#Modal
labelled_data = []
labelled_modal = []    
with open("resources/tagging_lexicons/modal.txt", 'r', encoding="ISO-8859-1") as f:
    labelled_data = f.readlines()   
for pairs in labelled_data:
    labelled_modal += [(pairs[:-1].strip().split(" ")[0].strip().lower(),pairs[:-1].strip().split(" ")[1].strip())]

In [10]:
word_lists_dict = {'LM_pos' :labelled_positive,'LM_neg' : labelled_negative ,'LagInd' : labelled_lagging ,'LeadInd' : labelled_leading ,'dir:UP' : [x[0] for x in labelled_direction if x[1] == 'UP'],'dir:DOWN': [x[0] for x in labelled_direction if x[1] == 'DOWN'], 'constrain' : final_labelled_constraining,'uncertain' : final_labelled_uncertainty,'mod:weak':[x[0] for x in labelled_modal if x[1] == 'WEAK'],'mod:medium': [x[0] for x in labelled_modal if x[1] == 'MEDIUM'],'mod:strong' : [x[0] for x in labelled_modal if x[1] == 'STRONG'] }

In [11]:
# need to modify exisitng dictionary for new tagging function
def new_tag_data_structure(tags_dict):
    new_tags = {}
    for category in tags_dict:
        temp_dict = {}
        first_words_list = [i.split(' ') for i in  tags_dict[category]]
        first_words_set = set([i[0] for i in  first_words_list])
        for i in first_words_set:
            for j in first_words_list:
                if j[0] == i:
                    if len(j) == 1:
                        temp_dict[i] = [[]]
                    else:
                        if i not in temp_dict:
                            temp_dict[i] = []
                        temp_dict[i].append(j[1:])
        new_tags[category] = temp_dict
    return new_tags

In [12]:
word_lists_dict = new_tag_data_structure(word_lists_dict)

In [15]:
# dumping lexicons on pkl format
import pickle
#dumping apriori rules in pkl format
pickling_on_ = open("resources/pickle/word_list_dict.pkl","wb")
pickle.dump(word_lists_dict, pickling_on_)
pickling_on_.close()

## Importing Libraries for Sentence2Vec Training

In [15]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
import re
import string
import spacy
import operator
from ml_models.pdf_parser import *
from ml_models.pre_processing import *
nlp = spacy.load("en_core_web_sm")

## Load real Financial Earnings Call: Training Data for Sent2Vec

link to google drive containing pdfs:
https://drive.google.com/drive/folders/19t-40BOKHhtgZcGYJKbewygx0YF-vo2n?usp=sharing

In [16]:
## Copy all the working pdf files on the drive to "app/resources/train_pdfs" folder(create this folder)
import os
raw_parsed_output = []
for filename in os.listdir("resources/train_pdfs/"):
    if filename.endswith(".pdf"): 
        print(filename)
        raw_parsed_output.append(pdf_parse(os.path.join("resources/train_pdfs/", filename)))

BNS-Q3F17.pdf


ERROR:root:company initial used as company name to divide sections
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/pdf_parser.py", line 195, in management_discussion
    [re.search(".*(?=" + company_name + ")", paragraph).group(0).strip()]
AttributeError: 'NoneType' object has no attribute 'group'
ERROR:root:unable to seperate speaker title with text in second format
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/pdf_parser.py", line 206, in management_discussion
    re.search("^(.*?)" + company_initial, paragraph).group(0).strip()[: -len(company_initial)]]
AttributeError: 'NoneType' object has no attribute 'group'
ERROR:root:Cannot extract name and title from Operator: Our first question comes from Meny Grauman with Cormark Securities
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/pdf_parser.py", line 163, in role
    name_title = (text[:min(co

ERROR:root:Cannot extract name and title from A [ph] Thanks (41:37).
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/pdf_parser.py", line 263, in first_occur
    type_pos = min(first_occuring.items(), key=lambda x: x[1])
ValueError: min() arg is an empty sequence
ERROR:root:unble to extract qna information from the QnA text block
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/pdf_parser.py", line 309, in qna_section
    qna_content += [qna_text_analysis(qna_text, executive_titles)]
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/pdf_parser.py", line 289, in qna_text_analysis
    company = text_qna[text_qna.find(",") + 1: type_index[1]].strip()
TypeError: slice indices must be integers or None or have an __index__ method
ERROR:root:Cannot extract name and title from Operator: Our next question comes from Scott Chan with Canaccord Genuity
Traceback (most recent call last):
  File "/

TD-Q4F18.pdf


ERROR:root:Cannot extract name and title from Operator: Thank you. [Operator Instructions] We'll take our first question from Gabriel Dechaine. Please go ahead
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/pdf_parser.py", line 163, in role
    name_title = (text[:min(counter)].strip(), text[min(counter):].strip())
ValueError: min() arg is an empty sequence
ERROR:root:Cannot extract name and title from Operator: Thank you. [Operator Instructions] We'll take our first question from Gabriel Dechaine. Please go ahead.
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/pdf_parser.py", line 263, in first_occur
    type_pos = min(first_occuring.items(), key=lambda x: x[1])
ValueError: min() arg is an empty sequence
ERROR:root:unble to extract qna information from the QnA text block
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/pdf_parser.py", line 309, 

...

ERROR:root:Cannot extract name and title from Operator: Thank you. [Operator Instructions] We'll take our first question from Ebrahim Poonawala with Bank of America Merrill Lynch
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/pdf_parser.py", line 163, in role
    name_title = (text[:min(counter)].strip(), text[min(counter):].strip())
ValueError: min() arg is an empty sequence
ERROR:root:Cannot extract name and title from Operator: Thank you. [Operator Instructions] We'll take our first question from Ebrahim Poonawala with Bank of America Merrill Lynch.
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/pdf_parser.py", line 263, in first_occur
    type_pos = min(first_occuring.items(), key=lambda x: x[1])
ValueError: min() arg is an empty sequence
ERROR:root:unble to extract qna information from the QnA text block
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdon

ERROR:root:unble to extract qna information from the QnA text block
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/pdf_parser.py", line 309, in qna_section
    qna_content += [qna_text_analysis(qna_text, executive_titles)]
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/pdf_parser.py", line 289, in qna_text_analysis
    company = text_qna[text_qna.find(",") + 1: type_index[1]].strip()
TypeError: slice indices must be integers or None or have an __index__ method
ERROR:root:Cannot extract name and title from Operator: And we'll take our next question from Steve Theriault with Eight Capital
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/pdf_parser.py", line 163, in role
    name_title = (text[:min(counter)].strip(), text[min(counter):].strip())
ValueError: min() arg is an empty sequence
ERROR:root:Cannot extract name and title from Operator: And we'll take our next question from Ste

TD-Q1F19.pdf


ERROR:root:Cannot extract name and title from Operator: Certainly. Thank you. [Operator Instructions] The first question is from Sumit Malhotra with Scotia Capital. Please go ahead
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/pdf_parser.py", line 163, in role
    name_title = (text[:min(counter)].strip(), text[min(counter):].strip())
ValueError: min() arg is an empty sequence
ERROR:root:Cannot extract name and title from Operator: Certainly. Thank you. [Operator Instructions] The first question is from Sumit Malhotra with Scotia Capital. Please go ahead.
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/pdf_parser.py", line 263, in first_occur
    type_pos = min(first_occuring.items(), key=lambda x: x[1])
ValueError: min() arg is an empty sequence
ERROR:root:unble to extract qna information from the QnA text block
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthi

ERROR:root:unble to extract qna information from the QnA text block
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/pdf_parser.py", line 309, in qna_section
    qna_content += [qna_text_analysis(qna_text, executive_titles)]
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/pdf_parser.py", line 289, in qna_text_analysis
    company = text_qna[text_qna.find(",") + 1: type_index[1]].strip()
TypeError: slice indices must be integers or None or have an __index__ method
ERROR:root:Cannot extract name and title from Operator: Thank you. This concludes the question-and-answer session. I would now like to turn the meeting back over to Bharat Masrani
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/pdf_parser.py", line 163, in role
    name_title = (text[:min(counter)].strip(), text[min(counter):].strip())
ValueError: min() arg is an empty sequence
ERROR:root:Cannot extract name and title from 

In [17]:
# Number of PDFs going to be used to train the word2vec
len(raw_parsed_output)

45

In [18]:
files = []
sections = []
for file in raw_parsed_output:
    files.append(file)
    sections += [(md['Text'], 'mds') for md in file["management_discussion_section"]]
    sections += [ (i['Question_text'], 'question') for i in file['question_and_answers']]
    sections += [ (j['A_text'], 'answer') for i in file['question_and_answers'] for j in i['Answer']] 
    try:
        sections += [ (file['final_remarks']['Text'], 'cr') ]
    except:
        pass

In [19]:
sections_df = pd.DataFrame(sections, columns = ['text', 'section'])
sections_df.head()

,text,section
0,"Thank you, Adam, and good morning, everyone. ...",mds
1,"Thanks, Brian. I will begin on slide 6, which...",mds
2,"Thank you, Sean. I'll start on slide 13. We c...",mds
3,"Thank you, Daniel. I'd like to close by highl...",mds
4,"Thanks, Brian. That concludes our prepared re...",mds


In [20]:
sections_df.shape

(3542, 2)

In [22]:
import pickle
training_s2v_dataset = sections_df['text']
pickling_on_ = open("resources/pickle/sent2vec_training_data.pkl","wb")
pickle.dump(training_s2v_dataset, pickling_on_)
pickling_on_.close()

# Training Sent2Vec

In [1]:
pickle_in = open("../resources/pickle/sent2vec_training_data.pkl","rb")
training_s2v_dataset = pickle.load(pickle_in)

In [23]:
from ml_models.sent2vec import Sent2Vec
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [24]:
# Default parameters size=50, window=5, min_count=5, workers=4, sg = 1, epochs= 50
s2v = Sent2Vec(size=50, window=7, min_count=2, workers=8, sg = 1, epochs= 10, stopwords = set(stopwords.words('english')), stemmer= PorterStemmer())

In [25]:
# train function trains the model and also saves it to pickle
s2v.train(training_s2v_dataset)


ERROR:root:could not find a match with word in lexicon, checking next lexicon word
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/apriori.py", line 68, in replace_words_with_tags
    if tokenize_text[inner_counter] == word:
IndexError: list index out of range
ERROR:root:could not find a match with word in lexicon, checking next lexicon word
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/apriori.py", line 68, in replace_words_with_tags
    if tokenize_text[inner_counter] == word:
IndexError: list index out of range
ERROR:root:could not find a match with word in lexicon, checking next lexicon word
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/apriori.py", line 68, in replace_words_with_tags
    if tokenize_text[inner_counter] == word:
IndexError: list index out of range
ERROR:root:could not find a match with word in lexicon, checking next lexico

ERROR:root:could not find a match with word in lexicon, checking next lexicon word
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/apriori.py", line 68, in replace_words_with_tags
    if tokenize_text[inner_counter] == word:
IndexError: list index out of range
ERROR:root:could not find a match with word in lexicon, checking next lexicon word
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/apriori.py", line 68, in replace_words_with_tags
    if tokenize_text[inner_counter] == word:
IndexError: list index out of range
ERROR:root:could not find a match with word in lexicon, checking next lexicon word
Traceback (most recent call last):
  File "/Users/davistanugraha/Desktop/getthisdone/ml_models/apriori.py", line 68, in replace_words_with_tags
    if tokenize_text[inner_counter] == word:
IndexError: list index out of range
ERROR:root:could not find a match with word in lexicon, checking next lexico

In [26]:
s2v.transform_text("""That's liquidity we hold for a rainy day. So it's just excess liquidity we keep from a risk management perspective.
I wouldn't expect that to grow at a higher rate than the overall balance sheet going forward.
Thank you, operator, and thank you, all, for your questions. I will close with a quick comment to reinforce the confidence that we've got in the momentum in all of our businesses. The bank is strong, diversified, and growing. And as you heard today, we're executing against the strategies that position us to grow in our target markets which is leading to accelerated growth and improving efficiency. So we're on track to achieve the financial targets we set out at the beginning of the year and sustainable growth over the longer term.  Thank you, everyone, for your time on the call today we look forward to speaking to you again at our Investor Day in October.""")

array([-0.11223785, -0.11779752, -0.3735351 , -0.28228015,  0.17414626,
        0.11614864,  0.03636776,  0.32569128, -0.30084515,  0.21774977,
       -0.04691575,  0.1563455 , -0.00620037, -0.20708863, -0.06807388,
       -0.1224912 , -0.2824294 , -0.09484794, -0.21068986,  0.12078916,
        0.02331301,  0.19357456,  0.04604859,  0.1278233 ,  0.24560714,
       -0.2698469 ,  0.06597026, -0.16282628, -0.21102527, -0.23552911,
       -0.11553819,  0.3524071 , -0.0782838 , -0.02429141, -0.05367053,
       -0.1419052 , -0.5174197 , -0.19730406,  0.18576044,  0.2095271 ,
        0.28321105, -0.03676943,  0.20812783, -0.16983698, -0.3047352 ,
       -0.30179802,  0.05991584, -0.21690767,  0.11209837, -0.08246278],
      dtype=float32)

In [ ]:
#storing sentence2vec model
#this sentence2vec saved class will also be used for word2vec transformation for the CNN method.
pickling_on_ = open("../resources/pickle/sent2vec.pkl","wb")
pickle.dump(s2v, pickling_on_)
pickling_on_.close()